Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [0]:
from google.colab import files
uploaded = files.upload()

Saving notMNIST.pickle to notMNIST.pickle


In [0]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [0]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [0]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [0]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [0]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
   
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.552683
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.129061
Minibatch accuracy: 75.0%
Validation accuracy: 53.8%
Minibatch loss at step 100: 1.719806
Minibatch accuracy: 56.2%
Validation accuracy: 72.7%
Minibatch loss at step 150: 1.127680
Minibatch accuracy: 68.8%
Validation accuracy: 70.8%
Minibatch loss at step 200: 1.006077
Minibatch accuracy: 68.8%
Validation accuracy: 76.2%
Minibatch loss at step 250: 0.412526
Minibatch accuracy: 87.5%
Validation accuracy: 79.8%
Minibatch loss at step 300: 0.663652
Minibatch accuracy: 81.2%
Validation accuracy: 79.5%
Minibatch loss at step 350: 0.292112
Minibatch accuracy: 93.8%
Validation accuracy: 80.7%
Minibatch loss at step 400: 0.529483
Minibatch accuracy: 87.5%
Validation accuracy: 80.7%
Minibatch loss at step 450: 0.466049
Minibatch accuracy: 87.5%
Validation accuracy: 81.4%
Minibatch loss at step 500: 0.174193
Minibatch accuracy: 93.8%
Validation accuracy: 81.5%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [0]:
batch_size = 16
patch_size = 5 #kernel size
image_size = 28
depth = 16
num_hidden = 64

graph = tf.Graph()
with graph.as_default():
    #placeholder
    tf_train_dataset = tf.placeholder(dtype = tf.float32, shape = (batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(dtype = tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Weight
    layer1_weight = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev = 0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    layer2_weight = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev = 0.1))
    layer2_biases = tf.Variable(tf.zeros([depth]))
    
    layer3_weight = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev = 0.1))
    layer3_biases = tf.Variable(tf.zeros([num_hidden]))

    layer4_weight = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev = 0.1))
    layer4_biases = tf.Variable(tf.zeros([num_labels]))
    
    #compute
    def model(data):
        conv = tf.nn.conv2d(input = data, filter = layer1_weight, strides = [1,1,1,1], padding = 'SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        pooling = tf.nn.max_pool(value = hidden, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID')
        
        conv = tf.nn.conv2d(input = pooling, filter = layer2_weight, strides = [1,1,1,1], padding = 'SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        pooling = tf.nn.max_pool(value = hidden, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID')
        
        shape = pooling.get_shape().as_list()
        reshape = tf.reshape(pooling, [shape[0], shape[1]*shape[2]*shape[3]])
        hidden = tf.matmul(reshape, layer3_weight) + layer3_biases
        
        return tf.matmul(hidden, layer4_weight) + layer4_biases
        
        
    logits = model(tf_train_dataset)
    
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))
    
    #optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    #predict
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))
    

Instructions for updating:
Colocations handled automatically by placer.


##### Run session

In [0]:
num_step = 1001

with tf.Session(graph = graph) as session:
    tf.global_variables_initializer().run()
    print('Init variable')
    
    for step in range(num_step):
        offset = (batch_size * step) % (train_dataset.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset:batch_data, tf_train_labels: batch_labels}
        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict = feed_dict)
        
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
        
    
    print('Test accuracy: %f' % accuracy(test_prediction.eval(), test_labels))

Init variable
Minibatch loss at step 0: 2.232879
Minibatch accuracy: 18.8%
Validation accuracy: 14.2%
Minibatch loss at step 50: 0.924045
Minibatch accuracy: 75.0%
Validation accuracy: 65.0%
Minibatch loss at step 100: 1.388810
Minibatch accuracy: 62.5%
Validation accuracy: 76.2%
Minibatch loss at step 150: 0.804022
Minibatch accuracy: 81.2%
Validation accuracy: 79.7%
Minibatch loss at step 200: 1.139033
Minibatch accuracy: 68.8%
Validation accuracy: 81.0%
Minibatch loss at step 250: 0.450202
Minibatch accuracy: 93.8%
Validation accuracy: 81.4%
Minibatch loss at step 300: 0.699687
Minibatch accuracy: 81.2%
Validation accuracy: 82.7%
Minibatch loss at step 350: 0.394132
Minibatch accuracy: 87.5%
Validation accuracy: 82.8%
Minibatch loss at step 400: 0.666866
Minibatch accuracy: 87.5%
Validation accuracy: 83.5%
Minibatch loss at step 450: 0.433673
Minibatch accuracy: 87.5%
Validation accuracy: 83.3%
Minibatch loss at step 500: 0.232778
Minibatch accuracy: 87.5%
Validation accuracy: 84.0%

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

Some notice:
+ Depth of conv layer
+ Number of step


In [0]:
batch_size = 16
depth = 64
depth1 = 64
depth2 = 64

patch_size = 5

num_hidden1 = 1024
num_hidden2 = 512

graph = tf.Graph()

with graph.as_default():
    
    # placeholder
    tf_train_dataset = tf.placeholder(dtype = tf.float32, shape = (batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(dtype = tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #weight
    #3 conv
    layer1_weight = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev = 0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    layer2_weight = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth1], stddev = 0.1))
    layer2_biases = tf.Variable(tf.zeros([depth1]))
    
    layer3_weight = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth1, depth2], stddev = 0.1))
    layer3_biases = tf.Variable(tf.zeros([depth2]))
    
    # 2 fc
    layer4_weight = tf.Variable(tf.truncated_normal([16*depth2, num_hidden1], stddev = 0.1))
    layer4_biases = tf.Variable(tf.zeros(num_hidden1))
    
    layer5_weight = tf.Variable(tf.truncated_normal([num_hidden1, num_hidden2], stddev = 0.1))
    layer5_biases = tf.Variable(tf.zeros(num_hidden2))
    
    # to output
    layer6_weight = tf.Variable(tf.truncated_normal([num_hidden2, num_labels], stddev = 0.1))
    layer6_biases = tf.Variable(tf.zeros(num_labels))
    
    #compute
    def model(data):
        #conv1
        conv = tf.nn.conv2d(input = data, filter = layer1_weight, strides = [1,1,1,1], padding = 'SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        pooling = tf.nn.max_pool(value = hidden, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')
        
        #conv2
        conv = tf.nn.conv2d(input = pooling, filter = layer2_weight, strides = [1,1,1,1], padding = 'SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        pooling = tf.nn.max_pool(value = hidden, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')
        
        #conv3
        conv = tf.nn.conv2d(input = pooling, filter = layer3_weight, strides = [1,1,1,1], padding = 'SAME')
        hidden = tf.nn.relu(conv + layer3_biases)
        pooling = tf.nn.max_pool(value = hidden, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')
        
        #fc1
        shape = pooling.get_shape().as_list()
        reshape = tf.reshape(pooling, [shape[0], shape[1]*shape[2]*shape[3]])
        hidden1 = tf.nn.relu(tf.matmul(reshape, layer4_weight) + layer4_biases)
        
        #dropout1
        hidden1 = tf.nn.dropout(hidden1, keep_prob = 0.5)
        
        
        #fc2 
        hidden2 = tf.nn.relu(tf.matmul(hidden1, layer5_weight) + layer5_biases)
        
        #dropout2
        hidden2 = tf.nn.dropout(hidden2, keep_prob = 0.5)
        
        #output
        return tf.matmul(hidden2, layer6_weight) + layer6_biases
    
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = logits, labels = tf_train_labels))
    
    
    #optimizer
    optimizer =tf.train.AdamOptimizer(1e-4).minimize(loss)
    #global_step = tf.Variable(0)
    #starter_learning_rate = 0.5
    #learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,500,0.96)
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    #prediction
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


##### Run session

In [0]:
num_step = 100001

with tf.Session(graph = graph) as session:
    tf.global_variables_initializer().run()
    print('Init variable')
    
    for step in range(num_step):
        offset = (batch_size * step) % (train_dataset.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset:batch_data, tf_train_labels: batch_labels}
        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict = feed_dict)
        
        if (step % 5000 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
        
    
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Init variable
Minibatch loss at step 0: 37.813530
Minibatch accuracy: 18.8%
Validation accuracy: 9.4%
Minibatch loss at step 5000: 0.247193
Minibatch accuracy: 87.5%
Validation accuracy: 82.6%
Minibatch loss at step 10000: 0.033372
Minibatch accuracy: 100.0%
Validation accuracy: 86.5%
Minibatch loss at step 15000: 0.583697
Minibatch accuracy: 81.2%
Validation accuracy: 88.0%
Minibatch loss at step 20000: 0.641897
Minibatch accuracy: 87.5%
Validation accuracy: 88.6%
Minibatch loss at step 25000: 0.292957
Minibatch accuracy: 87.5%
Validation accuracy: 89.3%
Minibatch loss at step 30000: 0.329637
Minibatch accuracy: 93.8%
Validation accuracy: 89.4%
Minibatch loss at step 35000: 0.196437
Minibatch accuracy: 93.8%
Validation accuracy: 90.0%
Minibatch loss at step 40000: 0.472378
Minibatch accuracy: 81.2%
Validation accuracy: 90.2%
Minibatch loss at step 45000: 0.245666
Minibatch accuracy: 87.5%
Validation accuracy: 90.3%
Minibatch loss at step 50000: 0.060818
Minibatch accuracy: 100.0%
Vali